# Объединение наборов данных: конкатенация и добавление в конец

In [1]:
import pandas as pd
import numpy as np

Для удобства опишем следующую функцию, создающую объект DataFrame определенной формы, которая нам пригодится в дальнейшем:

In [2]:
def make_df(cols, ind):
    """ Быстро создаем объект DataFrame """
    data = {c: [str(c) + str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)

In [3]:
# экземпляр DataFrame
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


## Напоминание: конкатенация массивов NumPy

In [4]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

Первый аргумент данной функции — список или кортеж объединяемых массивов. Кроме того, она принимает на входе ключевое слово axis, дающее возможность задавать ось, по которой будет выполняться конкатенация:

In [5]:
x = [[1, 2], [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

## Простая конкатенация с помощью метода pd.concat

In [6]:
# Сигнатура функции pd.concat в библиотеке Pandas v0.18
# pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)

Функцию pd.concat можно использовать для простой конкатенации объектов Series или DataFrame аналогично тому, как функцию np.concatenate() можно применять для простой конкатенации массивов:

In [7]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

Она также подходит для конкатенации объектов более высокой размерности, таких как DataFrame:

In [8]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
print(df1); print(df2); print(pd.concat([df1, df2]))

    A   B
1  A1  B1
2  A2  B2
    A   B
3  A3  B3
4  A4  B4
    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


По умолчанию конкатенация происходит в объекте DataFrame построчно, то есть axis=0. Аналогично функции np.concatenate() функция pd.concat() позволяет указывать ось, по которой будет выполняться конкатенация. Рассмотрим следующий пример:

In [9]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
print(df3); print(df4); print(pd.concat([df3, df4], axis = 1))

    A   B
0  A0  B0
1  A1  B1
    C   D
0  C0  D0
1  C1  D1
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


### Дублирование индексов

Важное различие между функциями np.concatenate() и pd.concat() состоит в том, что конкатенация из библиотеки Pandas сохраняет индексы, даже если в результате некоторые индексы будут дублироваться. Рассмотрим следующий пример:

In [10]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # дублируем индексы!
print(x, '\n')
print(y, '\n')
print(pd.concat([x, y]))

    A   B
0  A0  B0
1  A1  B1 

    A   B
0  A2  B2
1  A3  B3 

    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


Хотя в объектах DataFrame допустимо повторение индексов, подобный результат часто может быть нежелателен. Функция pd.concat() предоставляет нам несколько способов решения этой проблемы.

##### Прехват повторов как ошибки

Если вам нужно просто гарантировать, что индексы в возвращаемом функцией pd.concat() результате не перекрываются, можно задать флаг verify_integrity. В случае равного True значения этого флага конкатенация приведет к генерации ошибки при наличии дублирующихся индексов. Вот пример, в котором мы для большей ясности перехватываем и выводим в консоль сообщение об ошибке:

In [11]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print('ValueError:', e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


##### Игнорирование индекса 

Иногда индекс сам по себе не имеет значения и лучше его просто проигнорировать. Для этого достаточно установить флаг ignore_index. В случае равного True значения этого флага конкатенация приведет к созданию нового целочисленного индекса для итогового объекта Series:

In [12]:
print(x, '\n')
print(y, '\n')
print(pd.concat([x, y], ignore_index=True))

    A   B
0  A0  B0
1  A1  B1 

    A   B
0  A2  B2
1  A3  B3 

    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


##### Добавление ключей мультииндекса

Еще один вариант — воспользоваться параметром keys для задания меток для источников данных. Результатом будут иерархически индексированные ряды, содержащие данные:

In [13]:
print(x, '\n')
print(y, '\n')
print(pd.concat([x, y], keys=['x', 'y']))

    A   B
0  A0  B0
1  A1  B1 

    A   B
0  A2  B2
1  A3  B3 

      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3


Результат представляет собой мультииндексированный объект DataFrame

### Конкатенация с использованием соединений

В рассматриваемых примерах в основном производится конкатенация объектов DataFrame с общими названиями столбцов. На практике у данных из разных источников могут быть различные наборы имен столбцов. На этот случай у функции pd.concat() имеется несколько опций. Изучим объединение следующих двух объектов DataFrame, у которых столбцы (но не все!) называются одинаково:

In [14]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
print(df5, '\n')
print(df6, '\n')
print(pd.concat([df5, df6]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2 

    B   C   D
3  B3  C3  D3
4  B4  C4  D4 

     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


/srv/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


По умолчанию элементы, данные для которых отсутствуют, заполняются NA-значениями. Чтобы поменять это поведение, можно указать одну из нескольких опций для параметров join и join_axes функции конкатенации. По умолчанию соединение — объединение входных столбцов (join='outer'), но есть возможность поменять это поведение на пересечение столбцов с помощью опции join='inner':

In [15]:
print(df5, '\n')
print(df6, '\n')
print(pd.concat([df5, df6], join='inner'))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2 

    B   C   D
3  B3  C3  D3
4  B4  C4  D4 

    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4


Еще одна опция предназначена для указания явным образом индекса оставшихся столбцов с помощью аргумента join_axes, которому присваивается список объектов индекса. В данном случае мы указываем, что возвращаемые столбцы должны совпадать со столбцами первого из конкатенируемых объектов DataFrame:

In [16]:
print(df5, '\n')
print(df6, '\n')
print(pd.concat([df5, df6], join_axes=[df5.columns]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2 

    B   C   D
3  B3  C3  D3
4  B4  C4  D4 

     A   B   C
1   A1  B1  C1
2   A2  B2  C2
3  NaN  B3  C3
4  NaN  B4  C4


## Метод append()

Непосредственная конкатенация массивов настолько распространена, что в объекты Series и DataFrame был включен метод append(), позволяющий выполнить то же самое с меньшими усилиями. Например, вместо вызова pd.concat([df1, df2]) можно вызвать df1.append(df2):

In [17]:
print(df1, '\n')
print(df2, '\n')
print(df1.append(df2))

    A   B
1  A1  B1
2  A2  B2 

    A   B
3  A3  B3
4  A4  B4 

    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


Если необходимо выполнить несколько операций append, лучше создать список объектов DataFrame и передать их все сразу функции concat().

# Объединение наборов данных: слияние и соединение

Одно из важных свойств библиотеки Pandas — ее высокопроизводительные, выполняемые в оперативной памяти операции соединения и слияния. Если вы когда-либо работали с базами данных, вам должен быть знаком такой вид взаимодействия с данными. Основной интерфейс для них — функция pd.merge. Несколько примеров ее работы на практике мы рассмотрим далее.

## Реляционная алгебра

Реализованное в методе pd.merge поведение представляет собой подмножество того, что известно под названием «реляционная алгебра» (relational algebra). Реляционная алгебра — формальный набор правил манипуляции реляционными данными, формирующий теоретические основания для имеющихся в большинстве баз данных операций. Сила реляционного подхода состоит в предоставлении им нескольких простейших операций — своеобразных «кирпичиков» для построения более сложных операций над любым набором данных. При наличии эффективно реализованного в базе данных или другой программе подобного базового набора операций можно выполнять широкий диапазон весьма сложных составных операций.
Библиотека Pandas реализует несколько из этих базовых «кирпичиков» в функции pd.merge() и родственном ей методе join() объектов Series и DataFrame. Они обеспечивают возможность эффективно связывать данные из различных источников.

## Виды соединений

### Соединения "один к одному"

In [18]:
# исходные данные
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'], 
                    'hire_data': [2004, 2008, 2012, 2014]})
print(df1)
print()
print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

  employee  hire_data
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


Чтобы объединить эту информацию в один объект DataFrame, воспользуемся функцией pd.merge():

In [19]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_data
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


Функция pd.merge() распознает, что в обоих объектах DataFrame имеется столбец employee, и автоматически выполняет соединение, используя этот столбец в качестве ключа. Результатом слияния становится новый объект DataFrame, объединяющий информацию из двух входных объектов. Обратите внимание, что порядок записей в столбцах не обязательно сохраняется: в данном случае сортировка столбца employee различна в объектах df1 и df2 и функция pd.merge() обрабатывает эту ситуацию корректным образом. Кроме того, не забывайте, что слияние игнорирует индекс, за исключением особого случая слияния по индексу (рассмотрим далее)

### Соединение "многие-к-одному"

«Многие-к-одному» — соединения, при которых один из двух ключевых столбцов содержит дублирующиеся значения. В случае соединения «многие-к-одному» в итоговом объекте DataFrame эти дублирующиеся записи будут сохранены. Рассмотрим следующий пример соединения «многие-к-одному»:

In [20]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
print(df3)
print()
print(df4)
print()
print(pd.merge(df3, df4))

  employee        group  hire_data
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

  employee        group  hire_data supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve


В итоговом объекте DataFrame имеется дополнительный столбец с информацией о руководителе (supervisor) с повторением информации в одном или нескольких местах в соответствии с вводимыми данными.

### Соединение "многие-ко-многим"

Соединения «многие-ко-многим» семантически несколько более сложны, но тем не менее четко определены. Если столбец ключа как в левом, так и в правом массивах содержит повторяющиеся значения, результат окажется слиянием типа «многие-ко-многим». Рассмотрим следующий пример, в котором объект DataFrame отражает один или несколько навыков, соответствующих конкретной группе.
Выполнив соединение «многие-ко-многим», можно выяснить навыки каждого конкретного человека:

In [21]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting', 'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux', 'spreadsheets', 'organization']})
print(df1)
print()
print(df5)
print()
print(pd.merge(df1, df5))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization

  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization


## Задание ключа слияния

### Ключевое слово on

In [22]:
# нужно указать название ключевого столбца с помощью ключевого слова on, 
# в котором указывается название или список названий столбцов:
print(df1)
print()
print(df2)
print(pd.merge(df1, df2, on='employee'))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

  employee  hire_data
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014
  employee        group  hire_data
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


Этот параметр работает только в том случае, когда в левом и правом объектах DataFrame имеется указанное название столбца.

### Ключевые слова left_on и rigth_on

Иногда приходится выполнять слияние двух наборов данных с различными именами столбцов. Например, у нас может быть набор данных, в котором столбец для имени служащего называется Name, а не Employee. В этом случае можно воспользоваться ключевыми словами left_on и right_on для указания названий двух нужных столбцов:

In [23]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 'salary':[70000, 80000, 120000, 90000]})
print(df1)
print()
print(df3)
print(pd.merge(df1, df3, left_on='employee', right_on='name'))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000


Результат этой операции содержит избыточный столбец, который можно при желании удалить. Например, с помощью имеющегося в объектах DataFrame метода drop():

In [24]:
pd.merge(df1, df3, left_on='employee', right_on='name').drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### Ключевые слова left_index и right_index

Иногда удобнее вместо слияния по столбцу выполнить слияние по индексу. Допустим, у нас имеются следующие данные:

In [25]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
print(df1a)
print()
print(df2a)

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

          hire_data
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014


Можно использовать индекс в качестве ключа слияния путем указания в методе pd.merge() флагов left_index и/или right_index:

In [26]:
print(pd.merge(df1a, df2a, left_index=True, right_index=True))

                group  hire_data
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


Для удобства в объектах DataFrame реализован метод join(), выполняющий по умолчанию слияние по индексам:

In [27]:
print(df1a)
print()
print(df2a)
print(df1a.join(df2a))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

          hire_data
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
                group  hire_data
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


Если требуется комбинация слияния по столбцам и индексам, можно для достижения нужного поведения воспользоваться сочетанием флага left_index с параметром right_on или параметра left_on с флагом right_index:

In [28]:
print(df1a)
print()
print(df3)
pd.merge(df1a, df3, left_index=True, right_on='name')

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000


,group,name,salary
0,Accounting,Bob,70000
1,Engineering,Jake,80000
2,Engineering,Lisa,120000
3,HR,Sue,90000


Все эти параметры работают и в случае нескольких индексов и/или столбцов, синтаксис для этого интуитивно понятен.

## Задание операций над множествами для соединений

Во всех предыдущих примерах мы игнорировали один важный нюанс выполнения соединения — вид используемой при соединении операции алгебры множеств. Это играет важную роль в случаях, когда какое-либо значение есть в одном ключевом столбце, но отсутствует в другом. Рассмотрим следующий пример:

In [29]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'], 'food': ['fish', 'beans', 'bread']}, columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'], 'drink': ['wine', 'beer']}, columns=['name', 'drink'])
print(df6)
print()
print(df7)
print()
print(pd.merge(df6, df7))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

     name drink
0    Mary  wine
1  Joseph  beer

   name   food drink
0  Mary  bread  wine


Здесь мы слили воедино два набора данных, у которых совпадает только одна запись name: Mary. По умолчанию результат будет содержать пересечение двух входных множеств — внутреннее соединение (inner join). Можно указать это явным образом, с помощью ключевого слова how, имеющего по умолчанию значение 'inner':

In [30]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Левое соединение (left join) и правое соединение (right join) выполняют соединение по записям слева и справа соответственно. Например:

In [31]:
print(df6)
print()
print(df7)
print()
print(pd.merge(df6, df7, how='left'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

     name drink
0    Mary  wine
1  Joseph  beer

    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine


Строки результата теперь соответствуют записям в левом из входных объектов. Опция how='right' работает аналогичным образом.

In [32]:
print(df6)
print()
print(df7)
print()
print(pd.merge(df6, df7, how='right'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

     name drink
0    Mary  wine
1  Joseph  beer

     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer


Все эти опции можно непосредственно применять ко всем вышеописанным типам соединений.

### Пересекающиеся названия столбцов: ключевое слово suffixes

Может встретиться случай, когда в двух входных объектах присутствуют конфликтующие названия столбцов. Рассмотрим следующий пример:

In [33]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 'rank': [3, 1, 4, 2]})
print(df8)
print()
print(df9)
print()
print(pd.merge(df8, df9, on='name'))

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


Поскольку в результате должны были быть два конфликтующих названия столбцов, функция слияния автоматически добавила в названия суффиксы _x и _y, чтобы обеспечить уникальность названий столбцов результата. Если подобное поведение, принятое по умолчанию, неуместно, можно задать пользовательские суффиксы с помощью ключевого слова suffixes:

In [34]:
print(df8)
print()
print(df9)
print()
print(pd.merge(df8, df9, on='name', suffixes=['_L', '_R']))

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


Эти суффиксы будут работать для всех возможных вариантов соединений, в том числе и в случае нескольких пересекающихся по названию столбцов.

## Пример: данные по штатам США

пример с определенными данными по штатам США и их населению. Файлы данных можно найти по адресу http://github.com/jakevdp/data-USstates/

In [37]:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')
print(pop.head())
print()
print(areas.head())
print()
print(abbrevs.head()) # head выводит содержимое n строк файлов (по умолчанию 5)

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


Допустим, нам нужно на основе этой информации отсортировать штаты и территорию США по плотности населения в 2010 году. Информации для этого у нас достаточно, но для достижения цели придется объединить наборы данных.

Начнем со слияния «многие-ко-многим», которое позволит получить полное имя штата в объекте DataFrame для населения. Выполнить слияние нужно на основе столбца state/region объекта pop и столбца abbreviation объекта abbrevs. Мы воспользуемся опцией how='outer', чтобы гарантировать, что не упустим никаких данных из-за несовпадения меток.

In [66]:
merged = pd.merge(pop, abbrevs, how='outer', left_on='state/region', right_on='abbreviation') # объединить две таблицы, заполняя объектами NaN отсутствующие значения в итоговом объекте
merged = merged.drop('abbreviation', axis = 1)
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Следует проверить, не было ли каких-то несовпадений. Сделать это можно путем поиска строк с пустыми значениями:

In [67]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Часть информации о населении отсутствует, выясним, какая именно:

In [68]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


Похоже, что источник пустых значений по населению — Пуэрто-Рико, до 2000 года. Вероятно, это произошло из-за того, что необходимых данных не было в первоисточнике.
Мы видим, что некоторые из новых значений столбца state тоже пусты, а значит, в ключе объекта abbrevs отсутствовали соответствующие записи! Выясним, для каких территорий отсутствуют соответствующие значения:

In [69]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

Все понятно: наши данные по населению включают записи для Пуэрто-Рико (PR) и США в целом (USA), отсутствующие в ключе аббревиатур штатов. Исправим это, вставив соответствующие записи:

In [80]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


В столбце state больше нет пустых значений. Готово!
Теперь можно слить результат с данными по площади штатов с помощью аналогичной процедуры. После изучения имеющихся результатов становится понятно, что нужно выполнить соединение по столбцу state в обоих объектах:

In [82]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [90]:
final = pd.merge(merged, areas, on='state', how='left') # слить, количество строк как в левом объекте
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [91]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

В столбце area имеются пустые значения. Посмотрим, какие территории не были учтены:

In [97]:
final.loc[final['area (sq. mi)'].isnull(), 'state'].unique()

array(['United States'], dtype=object)

In [99]:
# еще один способ
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

Видим, что наш DataFrame-объект areas не содержит площадь США в целом. Мы могли бы вставить соответствующее значение (например, воспользовавшись суммой площадей всех штатов), но в данном случае мы просто удалим пустые значения, поскольку плотность населения США в целом нас сейчас не интересует:

In [101]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Теперь у нас есть все необходимые нам данные. Чтобы ответить на интересующий вопрос, сначала выберем часть данных, соответствующих 2010 году и всему населению. Воспользуемся функцией query() (для этого должен быть установлен пакет numexpr, см. раздел «Увеличение производительности библиотеки Pandas: eval() и query()» данной главы):

In [116]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Теперь вычислим плотность населения и выведем данные в соответствующем порядке. Начнем с переиндексации наших данных по штату, после чего вычислим результат:

In [117]:
# установим индекс state
data2010.set_index('state', inplace=True)

In [118]:
data2010.head()

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,total,2010,4785570.0,52423.0
Alaska,AK,total,2010,713868.0,656425.0
Arizona,AZ,total,2010,6408790.0,114006.0
Arkansas,AR,total,2010,2922280.0,53182.0
California,CA,total,2010,37333601.0,163707.0


In [122]:
density = data2010['population'] / data2010['area (sq. mi)']
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

Результат — список штатов США плюс Вашингтон (округ Колумбия) и Пуэрто-Рико, упорядоченный по плотности населения в 2010 году, в жителях на квадратную милю. Как видим, самая густонаселенная территория в этом наборе данных — Вашингтон (округ Колумбия); среди штатов же самый густонаселенный — Нью-Джерси.

In [123]:
# можно вывести окончание списка:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

Как видим, штатом с наименьшей плотностью населения, причем с большим отрывом от остальных, оказалась Аляска, насчитывающая в среднем одного жителя на квадратную милю.
Подобное громоздкое слияние данных — распространенная задача при ответе на вопросы, связанные с реальными источниками данных. Надеюсь, что этот пример дал вам представление, какими способами можно комбинировать вышеописанные инструменты, чтобы почерпнуть полезную информацию из данных!